In [1]:
import os
import h5py
import numpy 
import random
import math
import shutil
from tqdm import tqdm
from path import Path
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.offline as pyo

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

# Configuration

In [2]:
SAMPLE_POINTS = 2000
CLASSESS_CNT = 4

# Device

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
# device = "cpu"
print(f"Using {device} device")

Using cuda device


# Data Augmentation

In [4]:
class Normalize(object):
    def __call__(self, pointcloud):
        norm_pointcloud = pointcloud - numpy.mean(pointcloud, axis=0)
        norm_pointcloud /= numpy.max(numpy.linalg.norm(norm_pointcloud, axis=1))
        return  norm_pointcloud
    
class RandomNoise(object):
    def __call__(self, pointcloud):
        noise = numpy.random.normal(0, 0.02, (pointcloud.shape))
        noisy_pointcloud = pointcloud + noise
        return  noisy_pointcloud
    
class RandomScale(object):
    def __call__(self, pointcloud):
        s = numpy.random.uniform(0.9, 1.1, 3)
        rot_mat = numpy.array([[s[0], 0, 0],
                            [0, s[1], 0],
                            [0, 0, s[2]]])
        return numpy.matmul(pointcloud, rot_mat)

In [5]:
def default_transforms():
    return transforms.Compose([
        Normalize(),
        transforms.ToTensor()])

def default_transforms_no_normalize():
    return transforms.Compose([transforms.ToTensor()])

def training_transforms():
    return transforms.Compose([
        Normalize(),
        RandomNoise(),
        RandomScale(),
        transforms.ToTensor()])

def training_transforms_no_normalize():
    return transforms.Compose([
        RandomNoise(),
        RandomScale(),
        transforms.ToTensor()])

# Visualize Part Instance Utility Function

In [6]:
import plotly.graph_objects as go
import plotly.io as pio

def visualize_point_cloud(point_cloud, labels, export_svg=False, filename='point_cloud.svg'):
    trace = go.Scatter3d(
        x=point_cloud[:, 0],
        y=point_cloud[:, 1],
        z=point_cloud[:, 2],
        mode='markers',
        marker=dict(size=5, color=labels, colorscale='Viridis', opacity=0.5),
    )

    data = [trace]

    layout = go.Layout(
        scene=dict(
            xaxis=dict(visible=False, showbackground=False),
            yaxis=dict(visible=False, showbackground=False),
            zaxis=dict(visible=False, showbackground=False),
        ),
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)'
    )

    fig = go.Figure(data=data, layout=layout)
    
    if export_svg:
        pio.write_image(fig, filename, format='svg')
    else:
        pio.show(fig)

# Example usage
# visualize_point_cloud(point_cloud, labels, export_svg=True, filename='point_cloud.svg')


# Dataset

In [7]:
class GenerativeJawDataset(Dataset):
    def __init__(self, npy_file_path="/kaggle/input/generative-jaw/generative.npy", transform=training_transforms()):
        self.data = numpy.load(npy_file_path)
        self.transform = transform

    def __len__(self):
        return (self.data).shape[0]

    def __getitem__(self, idx):
        itemdata = self.data[idx]
        pointcloud = numpy.column_stack((itemdata[:,0], itemdata[:,1], itemdata[:,2]))
        label = itemdata[:,3] 
        d = 1  # d = 1, as it has annotations
        pointcloud = self.transform(pointcloud)
        pointcloud = pointcloud[0]
        
        lower = 1
        if idx >= 10: lower = 0
            
        assert pointcloud.shape[0] == SAMPLE_POINTS
        assert pointcloud.shape[1] == 3
        
        del itemdata
        torch.cuda.empty_cache()

        return pointcloud.type(torch.FloatTensor), torch.tensor(label).type(torch.LongTensor), torch.tensor(d).type(torch.LongTensor), torch.tensor(lower).type(torch.LongTensor)

In [8]:
class TestJawDataset(Dataset):
    def __init__(self, npy_file_path="/kaggle/input/real-jaw-2000-pointcloud-annotated-classbased/test_jaw.npy", transform=default_transforms()):
        self.data = numpy.load(npy_file_path)
        self.transform = transform

    def __len__(self):
        return 1 # TODO: Edit this later if we have more annotations

    def __getitem__(self, idx):
        itemdata = (self.data).astype(numpy.float32) # TODO: Edit this later if we have more annotations
        pointcloud = itemdata[:, :3]  # Extract x, y, z columns
        label = itemdata[:, 3] 
        d = 1  # d = 1, as it has annotations
        pointcloud = self.transform(pointcloud)
        pointcloud = pointcloud[0]
        
        if pointcloud.shape[0] > SAMPLE_POINTS:
            pointcloud = pointcloud[:2000]
            label = label[:2000]
        
        if pointcloud.shape[0] < SAMPLE_POINTS:
            pointcloud = torch.cat([pointcloud, pointcloud[-1].repeat(SAMPLE_POINTS-pointcloud.shape[0], 1)], dim=0)
            label = torch.cat([label, label[-1].repeat(SAMPLE_POINTS-label.shape[0], 1)], dim=0)
        
        assert pointcloud.shape[0] == SAMPLE_POINTS
        assert pointcloud.shape[1] == 3
        
        del itemdata
        torch.cuda.empty_cache()

        return pointcloud.type(torch.FloatTensor), torch.tensor(label).type(torch.LongTensor), torch.tensor(d).type(torch.LongTensor)

In [9]:
class RealJawDataset(Dataset):
    def __init__(self, data_dir="/kaggle/input/real-jaw-2000-pointcloud/RealJaw2000", transform=training_transforms()):
        self.data_dir = data_dir
        self.file_list = sorted(os.listdir(data_dir)[:20])
        self.transform = transform

    def __len__(self):
        return len(self.file_list)
    
    def load_xyz_file(self, file_path):
        with open(file_path, 'r') as f:
            data = numpy.loadtxt(f, delimiter=' ')
        return data

    def __getitem__(self, idx):
        file_path = os.path.join(self.data_dir, self.file_list[idx])

        data = self.load_xyz_file(file_path)

        # Load XYZ data from the file (assuming comma-delimited)
        pointcloud = numpy.column_stack((data[:,0], data[:,1], data[:,2])).astype(numpy.float32)

        # Create a fixed label (doesnt matter) and d (0)
        label = torch.zeros(SAMPLE_POINTS, dtype=torch.int64)
        d = torch.tensor(0, dtype=torch.int64)
        
        lower = idx%2
        
        pointcloud = self.transform(pointcloud)
        pointcloud = pointcloud[0]
        
        if pointcloud.shape[0] > SAMPLE_POINTS:
            pointcloud = pointcloud[:2000]
        
        if pointcloud.shape[0] < SAMPLE_POINTS :
            pointcloud = torch.cat([pointcloud, pointcloud[-1].repeat(SAMPLE_POINTS-pointcloud.shape[0], 1)], dim=0)
        
        assert pointcloud.shape[0] == SAMPLE_POINTS
        assert pointcloud.shape[1] == 3
        
        del file_path
        del data
        torch.cuda.empty_cache()

        return pointcloud.type(torch.FloatTensor),label, d,torch.tensor(lower).type(torch.LongTensor)

In [10]:
class TrainingDataset(Dataset):
    def __init__(self):
        self.generative_dataset = GenerativeJawDataset()
        self.real_dataset = RealJawDataset()

    def __len__(self):
        return len(self.generative_dataset) + len(self.real_dataset)

    def __getitem__(self, idx):
        if idx < len(self.real_dataset):
            return self.real_dataset[idx]
        else:
            return self.generative_dataset[idx-len(self.real_dataset)]

In [11]:
generative_dataset = GenerativeJawDataset()
test_dataset = TestJawDataset()
real_dataset = RealJawDataset()

# Visualize Real Data

In [12]:
data = real_dataset[18]
visualize_point_cloud(data[0], data[1])

# Visualize Real Annotated Test Data

In [13]:
data = test_dataset[0]

visualize_point_cloud(data[0], data[1])

In [14]:
# %pip install -U kaleido

# Visualize Generative Jaw Data With Annotations

In [15]:
data = generative_dataset[15]
visualize_point_cloud(data[0], data[1])

# Model: PointNet

In [16]:
class GradReverse(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x):
        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        return grad_output.neg()

def GRL(x):
    return GradReverse.apply(x)

In [17]:
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.utils.data
from torch.autograd import Variable
import numpy as np
import torch.nn.functional as F


class STN3d(nn.Module):
    def __init__(self, channel):
        super(STN3d, self).__init__()
        self.conv1 = torch.nn.Conv1d(channel, 64, 1)
        self.conv2 = torch.nn.Conv1d(64, 128, 1)
        self.conv3 = torch.nn.Conv1d(128, 256, 1)
        self.fc1 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64, 9)
        self.relu = nn.ReLU()

        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(256)
        self.bn4 = nn.BatchNorm1d(64)

    def forward(self, x):
        batchsize = x.size()[0]
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 256)

        x = F.relu(self.bn4(self.fc1(x)))
        x = self.fc3(x)

        iden = Variable(torch.from_numpy(np.array([1, 0, 0, 0, 1, 0, 0, 0, 1]).astype(np.float32))).view(1, 9).repeat(
            batchsize, 1)
        if x.is_cuda:
            iden = iden.cuda()
        x = x + iden
        x = x.view(-1, 3, 3)
        return x


class STNkd(nn.Module):
    def __init__(self, k=64):
        super(STNkd, self).__init__()
        self.conv1 = torch.nn.Conv1d(k, 64, 1)
        self.conv2 = torch.nn.Conv1d(64, 128, 1)
        self.conv3 = torch.nn.Conv1d(128, 256, 1)
        self.fc1 = nn.Linear(256, 128)
        self.fc2 = nn.Linear(128, k*k)
        self.relu = nn.ReLU()

        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(256)
        self.bn4 = nn.BatchNorm1d(128)

        self.k = k

    def forward(self, x):
        batchsize = x.size()[0]
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 256)

        x = F.relu(self.bn4(self.fc1(x)))
        x = self.fc2(x)

        iden = Variable(torch.from_numpy(np.eye(self.k).flatten().astype(np.float32))).view(1, self.k * self.k).repeat(
            batchsize, 1)
        if x.is_cuda:
            iden = iden.cuda()
        x = x + iden
        x = x.view(-1, self.k, self.k)
        return x

def feature_transform_reguliarzer(trans):
    d = trans.size()[1]
    I = torch.eye(d)[None, :, :]
    if trans.is_cuda:
        I = I.cuda()
    loss = torch.mean(torch.norm(torch.bmm(trans, trans.transpose(2, 1)) - I, dim=(1, 2)))
    return loss

In [18]:
class PointNetBackbone(nn.Module):
    def __init__(self):
        super(PointNetBackbone, self).__init__()
        self.stn = STN3d(3)
        self.conv1 = torch.nn.Conv1d(3, 64, 1)
        self.conv2 = torch.nn.Conv1d(64, 128, 1)
        self.conv3 = torch.nn.Conv1d(128, 256, 1)
        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(256)
        self.fstn = STNkd(k=64)

    def forward(self, x):
        B, D, N = x.size()
        trans = self.stn(x)
        x = x.transpose(2, 1)
        x = torch.bmm(x, trans)
        x = x.transpose(2, 1)
        x = F.relu(self.bn1(self.conv1(x)))

        trans_feat = self.fstn(x)
        x = x.transpose(2, 1)
        x = torch.bmm(x, trans_feat)
        x = x.transpose(2, 1)

        pointfeat = x
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.bn3(self.conv3(x))
        x = torch.max(x, 2, keepdim=True)[0]
        x = x.view(-1, 256)
        xx = x.view(-1, 256, 1).repeat(1, 1, N)
        return x, torch.cat([xx, pointfeat], 1), trans, trans_feat


In [19]:
class PointNetClassHead(nn.Module):
    def __init__(self, backbone=None, grl=True, k=2):
        super(PointNetClassHead, self).__init__()
        self.feat = backbone
        self.fc1 = nn.Linear(256, k)
        self.dropout = nn.Dropout(p=0.4)
        self.relu = nn.ReLU()
        self.grl = grl

    def forward(self, global_feature, combined_feature, trans, trans_feat):
        x = global_feature
        if self.grl == True: x = GRL(x)
        x = F.relu(self.dropout(self.fc1(x)))
        x = F.log_softmax(x, dim=1)
        return x, trans_feat

In [20]:
class PointNetSegHead(nn.Module):
    def __init__(self, backbone=None, part_num=4, grl=None):
        super(PointNetSegHead, self).__init__()

        self.feat = backbone
        
        self.head = nn.Sequential(
            nn.Conv1d(320, 128, kernel_size=1),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Conv1d(128, part_num, kernel_size=1)
        )

    def forward(self, global_feature, combined_feature, trans, trans_feat):
        x = self.head(combined_feature)
        x = x.transpose(2, 1)
        x = F.log_softmax(x, dim=1)
        return x, trans_feat

In [21]:
class PointNetDomainAdaptationModel(nn.Module):
    def __init__(self, m=CLASSESS_CNT, k1=2, k2=2):
        super(PointNetDomainAdaptationModel, self).__init__()

        self.m = m
        self.k1 = k1
        self.k2 =k2

        # Define the Fundamentals
        self.backbone = PointNetBackbone()
        self.domain_classifier = PointNetClassHead(backbone=self.backbone,grl=True,k=self.k1)
        self.lower_upper_classifier = PointNetClassHead(backbone=self.backbone,grl=True,k=self.k2)
        self.segmentation_head = PointNetSegHead(part_num=self.m)

    def forward(self, x):
        x_global, x_combined, trans, trans_feat = self.backbone(x)
        d_hat, d_trans_feat = self.domain_classifier(x_global, x_combined, trans, trans_feat)
        l_hat, l_trans_feat = self.lower_upper_classifier(x_global, x_combined, trans, trans_feat)
        seg_hat, seg_trans_feat = self.segmentation_head(x_global, x_combined, trans, trans_feat)
        return d_hat, d_trans_feat,l_hat, l_trans_feat, seg_hat, seg_trans_feat

# Finally

In [22]:
training_dataset = TrainingDataset()

In [23]:
len(training_dataset)

40

In [24]:
model = PointNetDomainAdaptationModel(m=CLASSESS_CNT,k1=2, k2=2).to(device)

In [25]:
# Count the number of trainable parameters
num_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Number of trainable parameters:", num_parameters)

# Calculate the model size in megabytes (MB)
model_size_mb = sum(p.numel() for p in model.parameters()) * 4 / (1024 ** 2)  # assuming 4 bytes per parameter
print("Model size:", model_size_mb, "MB")

# Calculate the size of the model's state dictionary in megabytes (MB)
model_state_dict_size_mb = sum(p.numel() for p in model.state_dict().values()) * 4 / (1024 ** 2)
print("Model state dictionary size:", model_state_dict_size_mb, "MB")

Number of trainable parameters: 752977
Model size: 2.8723793029785156 MB
Model state dictionary size: 8.326190948486328 MB


In [26]:
class PointNetSegLoss(torch.nn.Module):
    def __init__(self, mat_diff_loss_scale=0.001):
        super(PointNetSegLoss, self).__init__()
        self.mat_diff_loss_scale = mat_diff_loss_scale
        
        self.cross_entropy_loss = nn.CrossEntropyLoss()

    def forward(self, pred, target, trans_feat):
        loss = self.cross_entropy_loss(pred, target)
        mat_diff_loss = feature_transform_reguliarzer(trans_feat)
        total_loss = loss + mat_diff_loss * self.mat_diff_loss_scale
        return total_loss

In [37]:
class PointNetClassLoss(torch.nn.Module):
    def __init__(self, mat_diff_loss_scale=0.001):
        super(PointNetClassLoss, self).__init__()
        self.mat_diff_loss_scale = mat_diff_loss_scale
        
        self.cross_entropy_loss = nn.CrossEntropyLoss()

    def forward(self, pred, target, trans_feat):
        loss = self.cross_entropy_loss(pred, target)
        mat_diff_loss = feature_transform_reguliarzer(trans_feat)

        total_loss = loss + mat_diff_loss * self.mat_diff_loss_scale
        return total_loss

In [51]:
class TotalLoss(nn.Module):
    def __init__(self, mat_diff_loss_scale=0.001):
        super(TotalLoss, self).__init__()
        self.mat_diff_loss_scale = mat_diff_loss_scale
        
        # Lossess
        self.seg_loss = PointNetSegLoss(self.mat_diff_loss_scale)  
        self.class_loss = PointNetClassLoss(self.mat_diff_loss_scale)

    def forward(self, d_hat, d, d_A_feat,l_hat, l, l_A_feat, seg_hat, seg, seg_A_feat, domain_adapation_factor=0.5):        
        segmentation_loss = self.seg_loss(seg_hat, seg, seg_A_feat)
        domain_classification_loss = self.class_loss(d_hat, d, d_A_feat)
        label_classification_loss = self.class_loss(l_hat, l, l_A_feat)
        total_loss = d*segmentation_loss + domain_classification_loss*0.2 + label_classification_loss*0.1
        total_loss = total_loss.mean()
        return total_loss

In [52]:
def calculate_iou(predictions, targets):
    intersection = (predictions & targets).sum()
    union = (predictions | targets).sum()
    iou = intersection / union
    return iou

In [53]:
highest_iou = 0.0
def test(dataloader, model, loss_function):
    global highest_iou
    model.eval()
    total_iou = 0.0
    total_batches = len(dataloader)
    
    with torch.no_grad():
        for batch, data in enumerate(tqdm(dataloader)):
            pointcloud, label, d = (data[0].permute(0, 2, 1)).to(device), data[1].to(device), data[2].to(device)

            # Get Scores
            d_hat, d_trans_feat,l_hat, l_trans_feat, seg_hat, seg_trans_feat = model(pointcloud)

            predicted = torch.argmax(seg_hat, dim=2)

            # Calculate the IoU score for this batch
            iou = calculate_iou(predicted, label)

            total_iou += iou.item()
            
            # Clean memory
            del pointcloud
            del label
            del d
            del d_hat
            del d_trans_feat
            del l_hat
            del l_trans_feat
            del seg_hat
            del seg_trans_feat
            del iou
            del predicted
            torch.cuda.empty_cache()
            
    average_iou = total_iou / total_batches
    if average_iou >= highest_iou:
        highest_iou = average_iou
        torch.save(model.state_dict(), "/kaggle/working/model_state_highest.pth")
    print("Average IoU: " + str(average_iou))
    print("")
    print("Highest IoU: " + str(highest_iou))

In [54]:
def train(dataloader, model, optimizer, loss_function):
    torch.cuda.empty_cache()
    size = len(dataloader.dataset)
    model.train()
    loss_tot = 0.0
    num = 0
    for batch, data in enumerate(tqdm(dataloader)):
        pointcloud, label, d, l = (data[0].permute(0, 2, 1)).to(device), data[1].to(device), data[2].to(device), data[3].to(device)
    
        # Zeroing the gradients
        optimizer.zero_grad()

        number_of_parts = CLASSESS_CNT
        seg = torch.nn.functional.one_hot(label, number_of_parts).type(torch.FloatTensor).to(device)

        # Get Scores
        d_hat, d_trans_feat,l_hat, l_trans_feat, seg_hat, seg_trans_feat = model(pointcloud)

        # Calculate Loss
        loss = loss_function(d_hat, d, d_trans_feat,l_hat, l, l_trans_feat, seg_hat, seg, seg_trans_feat)

        # Backpropagation
        loss.backward()

        # Update
        optimizer.step()
        
        loss_tot += loss.item()
        num += 1
        
        # Clean memory
        del pointcloud
        del label
        del d
        del l
        del number_of_parts
        del seg
        del d_hat
        del d_trans_feat
        del l_hat
        del l_trans_feat
        del seg_hat
        del seg_trans_feat
        del loss
        torch.cuda.empty_cache()
        
    # loss_tot /= num
    print(f'training loss: {(loss_tot):>0.5f}')

In [55]:
def visualize_single_test_data(dataloader, model):
    model.eval()
    with torch.no_grad():
        for batch, data in enumerate(tqdm(dataloader)):
            pointcloud, label, d = (data[0].permute(0, 2, 1)).to(device), data[1].long().to(device), data[2].long().to(device)
            # Get Scores
            d_hat, d_trans_feat,l_hat, l_trans_feat, seg_hat, seg_trans_feat = model(pointcloud)
            predicted = torch.argmax(seg_hat, dim=2).cpu()
            
            pointcloud_2 = pointcloud.clone().permute(0,2,1).cpu()
            visualize_point_cloud(pointcloud_2[0], predicted[0])
            break

In [71]:
# Training Hyperparameters
epochs = 500
batch_size = 19
learning_rate = 0.000001
momentum=1.9
weight_decay=1.5

# Dataloader
training_data_loader = DataLoader(training_dataset, batch_size, shuffle = True)
testing_dataloader = DataLoader(test_dataset, batch_size, shuffle = False)

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

# Loss 
loss_function = TotalLoss(mat_diff_loss_scale=0.001).to(device)

# Scheduler
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=40, gamma=0.5)

# Training
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(training_data_loader, model, optimizer, loss_function)
    test(testing_dataloader, model,loss_function )
    torch.save(model.state_dict(), "/kaggle/working/model_state.pth")
print("Done!")


Epoch 1
-------------------------------


100%|██████████| 3/3 [00:00<00:00, 15.46it/s]


training loss: 6827.03931


100%|██████████| 1/1 [00:00<00:00, 177.12it/s]


Average IoU: 0.6459780335426331

Highest IoU: 0.7746073603630066
Epoch 2
-------------------------------


...

training loss: 3720.11650


100%|██████████| 1/1 [00:00<00:00, 214.21it/s]


Average IoU: 0.5426923632621765

Highest IoU: 0.7746073603630066
Epoch 56
-------------------------------


  0%|          | 0/3 [00:00<?, ?it/s]


KeyboardInterrupt: 

# Visualize Best Predictions

In [72]:
model.load_state_dict(torch.load("/kaggle/working/model_state_highest.pth"))

<All keys matched successfully>

In [73]:
visualize_single_test_data(testing_dataloader, model)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


In [ ]:
data = test_dataset[0]
visualize_point_cloud(data[0], data[1])